<a href="https://colab.research.google.com/github/gabrielfernandorey/EDVAI/blob/main/TP%20Final/Edvai_TP_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TRABAJO PRACTICO FINAL 
##### EDVAI

## Gabriel Rey 
##### Mayo 2023


In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('test_pyspark').getOrCreate()

In [4]:
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, BooleanType, FloatType, IntegerType, DoubleType, DateType
import pyspark.sql.functions as F
from pyspark.sql.functions import sum, col, desc, asc, count, countDistinct, round, max, min, avg
from pyspark.sql.functions import to_timestamp, date_format, to_date
from pyspark.sql.window import Window

from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, HasInputCols, HasOutputCols, Param, Params, TypeConverters
from pyspark import keyword_only
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml import Model
from pyspark.ml import Estimator

from datetime import datetime
import numpy as np
import pandas as pd

### Importamos datos

In [5]:
!wget https://data-engineer-edvai.s3.amazonaws.com/2021-informe-ministerio.csv
!wget https://data-engineer-edvai.s3.amazonaws.com/202206-informe-ministerio.csv
!wget https://data-engineer-edvai.s3.amazonaws.com/aeropuertos_detalle.csv

--2023-05-17 17:11:07--  https://data-engineer-edvai.s3.amazonaws.com/2021-informe-ministerio.csv
Resolving data-engineer-edvai.s3.amazonaws.com (data-engineer-edvai.s3.amazonaws.com)... 54.231.170.145, 3.5.3.152, 54.231.225.153, ...
Connecting to data-engineer-edvai.s3.amazonaws.com (data-engineer-edvai.s3.amazonaws.com)|54.231.170.145|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32322556 (31M) [text/csv]
Saving to: ‘2021-informe-ministerio.csv.2’

2021-informe-minist 100%[===================>]  30.82M  89.0MB/s    in 0.3s    

2023-05-17 17:11:08 (89.0 MB/s) - ‘2021-informe-ministerio.csv.2’ saved [32322556/32322556]

--2023-05-17 17:11:08--  https://data-engineer-edvai.s3.amazonaws.com/202206-informe-ministerio.csv
Resolving data-engineer-edvai.s3.amazonaws.com (data-engineer-edvai.s3.amazonaws.com)... 54.231.170.145, 3.5.3.152, 54.231.225.153, ...
Connecting to data-engineer-edvai.s3.amazonaws.com (data-engineer-edvai.s3.amazonaws.com)|54.231.170.145|:4

In [6]:
df_2021 = pd.read_csv("/content/2021-informe-ministerio.csv", sep=';')

In [98]:
df_2022 = pd.read_csv("/content/202206-informe-ministerio.csv", sep=';')

<ipython-input-98-bcad1b5984a1>:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_2022 = pd.read_csv("/content/202206-informe-ministerio.csv", sep=';')


In [99]:
df_aeropuertos = pd.read_csv("/content/aeropuertos_detalle.csv", sep=';')

### EDA

In [100]:
df_2021

,Fecha,Hora UTC,Clase de Vuelo (todos los vuelos),Clasificación Vuelo,Tipo de Movimiento,Aeropuerto,Origen / Destino,Aerolinea Nombre,Aeronave,Pasajeros,Calidad dato
0,01/01/2021,00:02,Vuelo Privado con Matrícula Nacional,Domestico,Despegue,PAR,ROS,0,PA-PA-28-181,0,DEFINITIVO
1,01/01/2021,00:24,Regular,Domestico,Aterrizaje,EZE,GRA,AEROLINEAS ARGENTINAS SA,BO-B737-8MB,70,DEFINITIVO
2,01/01/2021,00:26,Regular,Domestico,Aterrizaje,EZE,ECA,AEROLINEAS ARGENTINAS SA,BO-737-800,70,DEFINITIVO
3,01/01/2021,00:29,Regular,Domestico,Aterrizaje,EZE,SAL,AEROLINEAS ARGENTINAS SA,BO-B-737-76N,12,DEFINITIVO
4,01/01/2021,00:37,Regular,Domestico,Aterrizaje,EZE,TUC,AEROLINEAS ARGENTINAS SA,EMB-ERJ190100IGW,26,DEFINITIVO
...,...,...,...,...,...,...,...,...,...,...,...
328131,31/12/2021,23:44,Regular,Domestico,Aterrizaje,JUJ,AER,FB LÍNEAS AÉREAS - FLYBONDI,BO-737-8Q8,96,DEFINITIVO
328132,31/12/2021,23:45,Regular,Domestico,Despegue,AER,CBA,FB LÍNEAS AÉREAS - FLYBONDI,BO-737-8Q8,88,DEFINITIVO
328133,31/12/2021,23:48,Regular,Domestico,Despegue,ROS,AER,AEROLINEAS ARGENTINAS SA,BO-737-800,10,DEFINITIVO
328134,31/12/2021,23:55,Regular,Internacional,Aterrizaje,EZE,SCEL,SKY AIRLINE S.A.,0,137,DEFINITIVO


In [101]:
df_2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328136 entries, 0 to 328135
Data columns (total 11 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   Fecha                              328136 non-null  object
 1   Hora UTC                           328136 non-null  object
 2   Clase de Vuelo (todos los vuelos)  328136 non-null  object
 3   Clasificación Vuelo                328136 non-null  object
 4   Tipo de Movimiento                 328136 non-null  object
 5   Aeropuerto                         328136 non-null  object
 6   Origen / Destino                   328136 non-null  object
 7   Aerolinea Nombre                   328136 non-null  object
 8   Aeronave                           328136 non-null  object
 9   Pasajeros                          328136 non-null  int64 
 10  Calidad dato                       328136 non-null  object
dtypes: int64(1), object(10)
memory usage: 27.5+ MB


In [102]:
# Validamos si hay valores nulos en pasajeros
df_2021[df_2021['Pasajeros'].isna()] 


,Fecha,Hora UTC,Clase de Vuelo (todos los vuelos),Clasificación Vuelo,Tipo de Movimiento,Aeropuerto,Origen / Destino,Aerolinea Nombre,Aeronave,Pasajeros,Calidad dato


In [103]:
df_2022

,Fecha,Hora UTC,Clase de Vuelo (todos los vuelos),Clasificación Vuelo,Tipo de Movimiento,Aeropuerto,Origen / Destino,Aerolinea Nombre,Aeronave,Pasajeros,Calidad dato
0,01/01/2022,00:01,Regular,Doméstico,Aterrizaje,AER,ECA,AEROLINEAS ARGENTINAS SA,BO-737-8SH,69,DEFINITIVO
1,01/01/2022,00:05,Regular,Doméstico,Aterrizaje,AER,SAL,AEROLINEAS ARGENTINAS SA,BO-B737-8,65,DEFINITIVO
2,01/01/2022,00:05,Regular,Doméstico,Despegue,IGU,AER,JETSMART AIRLINES S.A.,AIB-A320-232,41,DEFINITIVO
3,01/01/2022,00:09,Regular,Doméstico,Aterrizaje,AER,GAL,AEROLINEAS ARGENTINAS SA,BO-B737-81D,73,DEFINITIVO
4,01/01/2022,00:09,Regular,Internacional,Despegue,EZE,KDFW,AMERICAN AIRLINES INC.,0,261,DEFINITIVO
...,...,...,...,...,...,...,...,...,...,...,...
222922,30/06/2022,23:59,Regular,Doméstico,Despegue,DOZ,AER,AEROLINEAS ARGENTINAS SA,BO-B737-8SH,"83,5",PROVISORIO
222923,30/06/2022,16:26,Regular,Doméstico,Despegue,VIE,BAR,AEROLINEAS ARGENTINAS SA,EMB-ERJ190100IGW,"33,5",PROVISORIO
222924,30/06/2022,13:08,Regular,Doméstico,Aterrizaje,VIE,AER,AEROLINEAS ARGENTINAS SA,EMB-ERJ190100IGW,"45,5",PROVISORIO
222925,30/06/2022,01:33,Regular,Doméstico,Despegue,LAR,AER,AEROLINEAS ARGENTINAS SA,EMB-ERJ190100IGW,"47,5",PROVISORIO


In [104]:
# Validamos si hay valores nulos en pasajeros
df_2022[df_2022['Pasajeros'].isna()] 


,Fecha,Hora UTC,Clase de Vuelo (todos los vuelos),Clasificación Vuelo,Tipo de Movimiento,Aeropuerto,Origen / Destino,Aerolinea Nombre,Aeronave,Pasajeros,Calidad dato


In [105]:
df_aeropuertos

,local,oaci,iata,tipo,denominacion,coordenadas,latitud,longitud,elev,uom_elev,...,condicion,control,region,fir,uso,trafico,sna,concesionado,provincia,inhab
0,ACB,NaN,NaN,Aeródromo,CORONEL BOGADO/AGROSERVICIOS,"33°16'20""S 60°34'14""W",-60.570660,-33.272260,44.0,Metros,...,PRIVADO,NOCONTROL,RACE,SAEF,AEROAPP,Nacional,NO,NO,SANTA FÉ,NO
1,ACH,NaN,NaN,Aeródromo,GENERAL ACHA,"37°24' 6""S 64°36'49""W",-64.613510,-37.401640,277.0,Metros,...,PUBLICO,NOCONTROL,RACE,SAEF,CIVIL,Nacional,NO,NO,LA PAMPA,NO
2,ACM,NaN,NaN,Aeródromo,ARRECIFES/LA CURA MALAL,"34° 4'33""S 60° 8'30""W",-60.141700,-34.075740,37.0,Metros,...,PRIVADO,NOCONTROL,RACE,SAEF,CIVIL,Nacional,NO,NO,BUENOS AIRES,NO
3,ADO,SAWD,PUD,Aeródromo,PUERTO DESEADO,"47°44' 6""S 65°54'15""W",-65.904100,-47.735110,82.0,Metros,...,PUBLICO,AERADIO,RASU,SAVF,CIVIL,Nacional,NO,NO,SANTA CRUZ,NO
4,ADT,NaN,NaN,Aeródromo,BANDERA/AGROSERVICIOS DOÑA TERESA,"28°51'19""S 62°15'53""W",-62.264620,-28.855410,75.0,Metros,...,PRIVADO,NOCONTROL,RANO,SACF,AEROAPP,Nacional,NO,NO,SANTIAGO DEL ESTERO,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688,YOS,SAZH,OYO,Aeródromo,TRES ARROYOS,"38°23' 8""S 60°19'39""W",-60.327500,-38.385556,122.0,Metros,...,PUBLICO,NOCONTROL,RACE,SAEF,NaN,Nacional,NO,NO,BUENOS AIRES,NO
689,YPY,NaN,NaN,Aeródromo,YAPEYÚ,"29°21'32""S 56°47'45""W",-56.795730,-29.358770,53.0,Metros,...,PRIVADO,NOCONTROL,RANE,SARR,NaN,Nacional,NO,NO,CORRIENTES,NO
690,ZAP,SAHZ,APZ,Aeródromo,ZAPALA,"38°58'33""S 70° 6'48""W",-70.113464,-38.975886,1048.0,Metros,...,PUBLICO,NOCONTROL,RACE,SAEF,NaN,Nacional,NO,NO,NEUQUÉN,NO
691,ZLM,NaN,NaN,Aeródromo,BELL VILLE/LA ZULEMA,"32°29'23""S 62°40'17""W",-62.671389,-32.489722,129.0,Metros,...,PRIVADO,NOCONTROL,RANO,SACF,AEROAPP,Nacional,NO,NO,CÓRDOBA,NO


In [106]:
df_aeropuertos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 693 entries, 0 to 692
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   local          693 non-null    object 
 1   oaci           124 non-null    object 
 2   iata           90 non-null     object 
 3   tipo           693 non-null    object 
 4   denominacion   693 non-null    object 
 5   coordenadas    693 non-null    object 
 6   latitud        693 non-null    float64
 7   longitud       693 non-null    float64
 8   elev           693 non-null    float64
 9   uom_elev       693 non-null    object 
 10  ref            683 non-null    object 
 11  distancia_ref  688 non-null    float64
 12  direccion_ref  679 non-null    object 
 13  condicion      693 non-null    object 
 14  control        693 non-null    object 
 15  region         693 non-null    object 
 16  fir            693 non-null    object 
 17  uso            470 non-null    object 
 18  trafico   

In [107]:
# Validamos si hay valores nulos en distancia_ref
df_aeropuertos[df_aeropuertos['distancia_ref'].isna()] 

,local,oaci,iata,tipo,denominacion,coordenadas,latitud,longitud,elev,uom_elev,...,condicion,control,region,fir,uso,trafico,sna,concesionado,provincia,inhab
266,HET,NaN,NaN,Helipuerto,SANTIAGO DEL ESTERO/HELIPUERTO EDIFICIO TRIBUN...,"27°46'53""S 64°15'54""W",-64.265127,-27.781396,195.0,Metros,...,PRIVADO,NOCONTROL,RANO,SACF,NaN,Nacional,NO,NO,SANTIAGO DEL ESTERO,NO
299,HMG,NaN,NaN,Helipuerto,BUENOS AIRES / MADERO HARBOUR,"34°37'16""S 58°21'39""W",-58.360928,-34.620978,78.8,Metros,...,PRIVADO,NOCONTROL,RACE,SAEF,CIVIL,Nacional,NO,NO,CIUDAD AUTÓNOMA DE BUENOS AIRES,NO
307,HNH,NaN,NaN,Helipuerto,HELIPLATAFORMA NOBLE HOUSTON COLBERT,"53° 5'54""S 67°13'13""W",-67.220278,-53.098333,63.5,Metros,...,PRIVADO,NOCONTROL,RASU,SAVF,NaN,Nacional,NO,NO,TIERRA DEL FUEGO ANTÁRTIDA E ISLAS DEL ATLÁNTI...,NO
341,HSW,NaN,NaN,Helipuerto,HELIPLATAFORMA SWIBER PJW3000,"53° 4'35""S 67°57'50""W",-67.964004,-53.076495,36.0,Metros,...,PRIVADO,NOCONTROL,RASU,SAVF,NaN,Nacional,NO,NO,TIERRA DEL FUEGO ANTÁRTIDA E ISLAS DEL ATLÁNTI...,NO
439,MBI,SAWB,NaN,Aeródromo,VICECOMODORO MARAMBIO,"64°14'21""S 56°37'51""W",-56.630850,-64.239130,209.5,Metros,...,PUBLICO,CONTROL,RASU,SAVF,MIL,Nacional,NO,NO,TIERRA DEL FUEGO ANTÁRTIDA E ISLAS DEL ATLÁNTI...,NO


### SPARK



#### Transformaciones dataframes vuelos

In [113]:
schema = StructType([
            StructField("Fecha", StringType(), True),
            StructField("horaUTC", StringType(), True),
            StructField("clase_de_vuelo", StringType(), True),
            StructField("clasificacion_de_vuelo", StringType(), True),
            StructField("tipo_de_movimiento", StringType(), True),
            StructField("aeropuerto", StringType(), True),
            StructField("origen_destino", StringType(), True),
            StructField("aerolinea_nombre", StringType(), True),
            StructField("aeronave", StringType(), True),
            StructField("pasajeros", IntegerType(), True)
])

In [114]:
# Con esta alternativa los tipos de datos se importan de forma adecuada (siempre verificar!)
spk_2021 = spark.read.csv("/content/2021-informe-ministerio.csv", sep=';', schema=schema, header=True).withColumn("fecha", to_date("Fecha","dd/MM/yyyy"))

In [115]:
spk_2021

DataFrame[fecha: date, horaUTC: string, clase_de_vuelo: string, clasificacion_de_vuelo: string, tipo_de_movimiento: string, aeropuerto: string, origen_destino: string, aerolinea_nombre: string, aeronave: string, pasajeros: int]

In [116]:
spk_2021.show(10)

+----------+-------+--------------------+----------------------+------------------+----------+--------------+--------------------+----------------+---------+
|     fecha|horaUTC|      clase_de_vuelo|clasificacion_de_vuelo|tipo_de_movimiento|aeropuerto|origen_destino|    aerolinea_nombre|        aeronave|pasajeros|
+----------+-------+--------------------+----------------------+------------------+----------+--------------+--------------------+----------------+---------+
|2021-01-01|  00:02|Vuelo Privado con...|             Domestico|          Despegue|       PAR|           ROS|                   0|    PA-PA-28-181|        0|
|2021-01-01|  00:24|             Regular|             Domestico|        Aterrizaje|       EZE|           GRA|AEROLINEAS ARGENT...|     BO-B737-8MB|       70|
|2021-01-01|  00:26|             Regular|             Domestico|        Aterrizaje|       EZE|           ECA|AEROLINEAS ARGENT...|      BO-737-800|       70|
|2021-01-01|  00:29|             Regular|           

In [112]:
spk_2021

DataFrame[fecha: string, horaUTC: string, clase_de_vuelo: string, clasificacion_de_vuelo: string, tipo_de_movimiento: string, aeropuerto: string, origen_destino: string, aerolinea_nombre: string, aeronave: string, pasajeros: int, nueva: date]

In [117]:
spk_2022 = spark.read.csv("/content/202206-informe-ministerio.csv", sep=';', schema=schema, header=True).withColumn("fecha", to_date("Fecha","dd/MM/yyyy"))

In [118]:
spk_2022.show(5)

+----------+-------+--------------+----------------------+------------------+----------+--------------+--------------------+------------+---------+
|     fecha|horaUTC|clase_de_vuelo|clasificacion_de_vuelo|tipo_de_movimiento|aeropuerto|origen_destino|    aerolinea_nombre|    aeronave|pasajeros|
+----------+-------+--------------+----------------------+------------------+----------+--------------+--------------------+------------+---------+
|2022-01-01|  00:01|       Regular|             Doméstico|        Aterrizaje|       AER|           ECA|AEROLINEAS ARGENT...|  BO-737-8SH|       69|
|2022-01-01|  00:05|       Regular|             Doméstico|        Aterrizaje|       AER|           SAL|AEROLINEAS ARGENT...|   BO-B737-8|       65|
|2022-01-01|  00:05|       Regular|             Doméstico|          Despegue|       IGU|           AER|JETSMART AIRLINES...|AIB-A320-232|       41|
|2022-01-01|  00:09|       Regular|             Doméstico|        Aterrizaje|       AER|           GAL|AEROLINEA

#### Transformaciones dataframe aeropuerto

In [ ]:
schema2 = StructType([
            StructField("aeropuerto", StringType(), True),
            StructField("oac", StringType(), True),
            StructField("iata", StringType(), True),
            StructField("tipo", StringType(), True),
            StructField("denominacion", StringType(), True),
            StructField("coordenadas", StringType(), True),
            StructField("latitud", StringType(), True),
            StructField("longitud", StringType(), True),
            StructField("elev", FloatType(), True),
            StructField("uom_elev", StringType(), True),
            StructField("ref", StringType(), True),
            StructField("distancia_ref", FloatType(), True),
            StructField("condicion", StringType(), True),
            StructField("control", StringType(), True),
            StructField("region", StringType(), True),
            StructField("uso", StringType(), True),
            StructField("trafico", StringType(), True),
            StructField("sna", StringType(), True),
            StructField("concesionado", StringType(), True),
            StructField("provincia", StringType(), True)
])

In [ ]:
spk_aeropuertos = spark.read.csv("/content/aeropuertos_detalle.csv", sep=';', schema=schema2, header=True)

In [ ]:
spk_aeropuertos.show(5)

+----------+----+----+---------+--------------------+--------------------+------------+------------+-----+--------+--------------+-------------+---------+-------+---------+----+-------+-------+------------+---------+
|aeropuerto| oac|iata|     tipo|        denominacion|         coordenadas|     latitud|    longitud| elev|uom_elev|           ref|distancia_ref|condicion|control|   region| uso|trafico|    sna|concesionado|provincia|
+----------+----+----+---------+--------------------+--------------------+------------+------------+-----+--------+--------------+-------------+---------+-------+---------+----+-------+-------+------------+---------+
|       ACB|null|null|Aeródromo|CORONEL BOGADO/AG...|"33°16'20""S  60°...|-60.57066000|-33.27226000| 44.0|  Metros|Coronel Bogado|          6.0|       NE|PRIVADO|NOCONTROL|RACE|   SAEF|AEROAPP|    Nacional|       NO|
|       ACH|null|null|Aeródromo|        GENERAL ACHA|"37°24' 6""S  64°...|-64.61351000|-37.40164000|277.0|  Metros|  General Acha|  

### Union vuelos 2021-2022

In [48]:
spk_vuelos = spk_2021.union(spk_2022)

In [49]:
spk_vuelos.show(5)

+----------+-------+--------------------+----------------------+------------------+----------+--------------+--------------------+----------------+---------+
|     fecha|horaUTC|      clase_de_vuelo|clasificacion_de_vuelo|tipo_de_movimiento|aeropuerto|origen_destino|    aerolinea_nombre|        aeronave|pasajeros|
+----------+-------+--------------------+----------------------+------------------+----------+--------------+--------------------+----------------+---------+
|01/01/2021|  00:02|Vuelo Privado con...|             Domestico|          Despegue|       PAR|           ROS|                   0|    PA-PA-28-181|        0|
|01/01/2021|  00:24|             Regular|             Domestico|        Aterrizaje|       EZE|           GRA|AEROLINEAS ARGENT...|     BO-B737-8MB|       70|
|01/01/2021|  00:26|             Regular|             Domestico|        Aterrizaje|       EZE|           ECA|AEROLINEAS ARGENT...|      BO-737-800|       70|
|01/01/2021|  00:29|             Regular|           

In [50]:
spk_vuelos.count()

551063

In [18]:
spk_vuelos_nacionales = spk_vuelos.filter(spk_vuelos['clasificacion_de_vuelo'] == 'Domestico')

In [19]:
spk_vuelos_nacionales.count()

293069

Determinar la cantidad de vuelos entre las fechas 01/12/2021 y 31/01/2022

In [20]:
vuelos_nacionales_2021 = spk_vuelos_nacionales.filter(spk_vuelos_nacionales["fecha"] <= '31/01/2021').count()

In [21]:
vuelos_nacionales_2021

288085

In [51]:
spk_vuelos_nacionales.show(30)

+----------+-------+--------------------+----------------------+------------------+----------+--------------+--------------------+----------------+---------+
|     fecha|horaUTC|      clase_de_vuelo|clasificacion_de_vuelo|tipo_de_movimiento|aeropuerto|origen_destino|    aerolinea_nombre|        aeronave|pasajeros|
+----------+-------+--------------------+----------------------+------------------+----------+--------------+--------------------+----------------+---------+
|01/01/2021|  00:02|Vuelo Privado con...|             Domestico|          Despegue|       PAR|           ROS|                   0|    PA-PA-28-181|        0|
|01/01/2021|  00:24|             Regular|             Domestico|        Aterrizaje|       EZE|           GRA|AEROLINEAS ARGENT...|     BO-B737-8MB|       70|
|01/01/2021|  00:26|             Regular|             Domestico|        Aterrizaje|       EZE|           ECA|AEROLINEAS ARGENT...|      BO-737-800|       70|
|01/01/2021|  00:29|             Regular|           

Cantidad de pasajeros que viajaron en Aerolíneas Argentinas entre el 01/01/2021 y 30/06/2022

In [30]:
pasajeros_aerolineasargentinas = spk_vuelos_nacionales.filter(spk_vuelos_nacionales["aerolinea_nombre"] == "AEROLINEAS ARGENTINAS SA")

In [31]:
pasajeros_aerolineasargentinas.show(5)

+----------+-------+--------------+----------------------+------------------+----------+--------------+--------------------+----------------+---------+
|     fecha|horaUTC|clase_de_vuelo|clasificacion_de_vuelo|tipo_de_movimiento|aeropuerto|origen_destino|    aerolinea_nombre|        aeronave|pasajeros|
+----------+-------+--------------+----------------------+------------------+----------+--------------+--------------------+----------------+---------+
|01/01/2021|  00:24|       Regular|             Domestico|        Aterrizaje|       EZE|           GRA|AEROLINEAS ARGENT...|     BO-B737-8MB|       70|
|01/01/2021|  00:26|       Regular|             Domestico|        Aterrizaje|       EZE|           ECA|AEROLINEAS ARGENT...|      BO-737-800|       70|
|01/01/2021|  00:29|       Regular|             Domestico|        Aterrizaje|       EZE|           SAL|AEROLINEAS ARGENT...|    BO-B-737-76N|       12|
|01/01/2021|  00:37|       Regular|             Domestico|        Aterrizaje|       EZE|

In [32]:
total_pasajeros = pasajeros_aerolineasargentinas.agg(sum("pasajeros")).collect()[0][0]

In [33]:
total_pasajeros

4321079

Mostrar fecha, hora, código aeropuerto salida, ciudad de salida, código de aeropuerto de arribo, ciudad de arribo, y cantidad de pasajeros de cada vuelo, entre el 01/01/2022 y el 30/06/2022 ordenados por fecha de manera descendiente. 


In [57]:
spk_vuelos_nacionales.createOrReplaceTempView("vuelos_horarios")

In [35]:
spk_vuelos_nacionales

DataFrame[fecha: string, horaUTC: string, clase_de_vuelo: string, clasificacion_de_vuelo: string, tipo_de_movimiento: string, aeropuerto: string, origen_destino: string, aerolinea_nombre: string, aeronave: string, pasajeros: int]

In [55]:
spk_aeropuertos.createOrReplaceTempView("aeropuertos")

In [52]:
spk_aeropuertos

DataFrame[aeropuerto: string, oac: string, iata: string, tipo: string, denominacion: string, coordenadas: string, latitud: string, longitud: string, elev: float, uom_elev: string, ref: string, distancia_ref: float, condicion: string, control: string, region: string, uso: string, trafico: string, sna: string, concesionado: string, provincia: string]

In [96]:
consulta_sql = """
SELECT cast(to_date(vh.fecha, 'MM-dd-yyyy') AS date) as fecha, vh.horaUTC, vh.aeropuerto AS codigo_salida, a1.denominacion AS ciudad_salida, vh.origen_destino AS codigo_arribo, a2.denominacion AS ciudad_arribo, vh.pasajeros
FROM vuelos_horarios vh
JOIN aeropuertos a1 ON vh.aeropuerto = a1.aeropuerto
JOIN aeropuertos a2 ON vh.origen_destino = a2.aeropuerto
WHERE vh.fecha >= '2022/01/01'
ORDER BY vh.fecha DESC
"""

In [97]:
vuelos_ordenados = spark.sql(consulta_sql)

In [98]:
vuelos_ordenados.show(5)

+-----+-------+-------------+--------------------+-------------+--------------------+---------+
|fecha|horaUTC|codigo_salida|       ciudad_salida|codigo_arribo|       ciudad_arribo|pasajeros|
+-----+-------+-------------+--------------------+-------------+--------------------+---------+
| null|  00:04|          CBA|CÓRDOBA/ING. AER....|          BAR|SAN CARLOS DE BAR...|       71|
| null|  00:13|          AER|BUENOS AIRES/AERO...|          GRA|          RÍO GRANDE|       82|
| null|  00:07|          IGU|CATARATAS DEL  IG...|          AER|BUENOS AIRES/AERO...|       72|
| null|  00:09|          FDO|        SAN FERNANDO|          FDO|        SAN FERNANDO|        0|
| null|  00:10|          AER|BUENOS AIRES/AERO...|          POS|POSADAS/LIBERTADO...|       50|
+-----+-------+-------------+--------------------+-------------+--------------------+---------+
only showing top 5 rows

